# Week 6: Accessing satellite data from Google Earth Engine from Python

Individual learning outcomes: At the end of this week, all students should be able to access Sentinel-2 image composites from Google Earth Engine via the Python API, set up and submit a data query, download the data to Google Drive and Colab, and create a movie from a time series.

# Get a user account for Google Earth Engine

Before we begin, make sure to register for an account.

For registration follow the link to the Open Access Hub and register: https://signup.earthengine.google.com/#!/

In previous weeks, we had manually uploaded a Sentinel-2 image to our Google Drive directory.

Today, we want to access Sentinel-2 imagery from Google Earth Engine (GEE) and search for available images over an area of interest of our choice. GEE allows users to submit a processing request. This is different from just accessing data, as it allows the user to request image composites that are aggregated from several different individual image takes from different dates, and the user can define the area for the download.

# Accessing Sentinel-2 images

Workflow for this practical:
* Define an area of interest based on an ESRI shapefile
* Define a time window for our data search
* Set a maximum acceptable cloud cover for our search
* Use Google Earth Engine to make temporal composites of available images for selected spectral bands
* Download them to your Google Drive
* Reproject (warp) the images to the projection of the shapefile
* Plot maps of the images
* Make a movie for our area of interest


Connect to our Google Drive from Colab.

In [ ]:
# Load the Drive helper and mount your Google Drive as a drive in the virtual machine
from google.colab import drive
drive.mount('/content/drive')

Import required libraries

In [ ]:
#import required libraries
!pip install earthengine-api
!pip install requests
!pip install rasterio
!pip install geopandas
import geopandas as gpd
import rasterio
from rasterio import plot
from rasterio.plot import show_hist
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal, ogr
import json
import os
from os import listdir
from os.path import isfile, isdir, join
import math
from pprint import pprint
import shutil
import sys
import zipfile
import requests
import io
import webbrowser
import ee
%matplotlib inline

# Set up some directory paths on Google Drive
Modify these string variables to match your data directory structure if need be.

BEFORE YOU RUN THIS CELL, EDIT THE VARIABLE wd BELOW TO POINT TO YOUR DIRECTORY ON GOOGLE DRIVE

IMPORTANT: You must upload a shapefile of your area of interest to your Google Drive before running the next cell. Set the variable 'shapefile' below to point to this file. You can draw a polygon and save it as a shapefile on http://www.geojson.io.

In [ ]:
# set up your directories for the satellite data
# Note that we do all the downloading and data analysis on the temporary drive
#    on Colab. We will copy the output directory to our Google Drive at the end.
#    Colab has more disk space (about 40 GB free space) than Google Drive (15 GB).
#    However, the data on the Colab disk space are NOT kept when you log out.

# path to your Google Drive
# EDIT THIS LINE (/content/drive/My Drive is the top directory on Google Drive):
wd = "/content/drive/MyDrive/practicals20-21"
print("Connected to data directory: " + wd)

# path to your temporary drive on the Colab Virtual Machine
cd = "/content/work"

# directory for downloading the Sentinel-2 composites
# Note that we are using the 'join' function imported from the os library here
# It is an easy way of merging strings into a directory structure.
# It is clever and chooses the / or \ depending on whether you are on Windows or Linux.
downloaddir = join(cd, 'download') # where we save the downloaded images

# CAREFUL: This code removes the named directories and everything inside them to free up space
# Note: shutil provides a lot of useful functions for file and directory management
try:
  shutil.rmtree(downloaddir)
except:
  print(downloaddir + " not found.")

# create the new directories, unless they already exist
os.makedirs(cd, exist_ok=True)
os.makedirs(downloaddir, exist_ok=True)

print("Connected to Colab temporary data directory: " + cd)

print("\nList of contents of " + wd)
for f in sorted(os.listdir(wd)):
  print(f)

# Define our search parameters

You can modify some of the parameters and upload your own shapefile.

In [ ]:
# EDIT THE SEARCH OPTIONS BELOW

# YOU CAN PLACE A DIFFERENT SHAPEFILE ONTO YOUR GOOGLE DRIVE BUT MAKE SURE THAT
#    THE VARIABLE shapefile POINTS TO THE CORRECT FILE:
shapefile = join(wd, 'oakham', 'Polygons_small.shp') # ESRI Shapefile of the study area

# Define a date range for our search
datefrom = '2019-03-01' # start date for imagery search
dateto   = '2019-04-30' # end date for imagery search
time_range = [datefrom, dateto] # format as a list

# Define which cloud cover we accept in the images
clouds = 10 # maximum acceptable cloud cover in %

To make efficient use of Google Earth Engine from Python, we want to define some useful helper functions from https://climada-python.readthedocs.io/en/stable/tutorial/climada_util_earth_engine.html

In [ ]:
# Functions modified from climada.util.earth_engine module
def obtain_image_landsat_composite(collection, time_range, area):
    """ Selection of Landsat cloud-free composites in the Earth Engine library
    See also: https://developers.google.com/earth-engine/landsat

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        image_composite (ee.image.Image)
     """
    collection = ee.ImageCollection(collection)

    ## Filter by time range and location
    collection_time = collection.filterDate(time_range[0], time_range[1])
    image_area = collection_time.filterBounds(area)
    image_composite = ee.Algorithms.Landsat.simpleComposite(image_area, 75, 3)
    return image_composite

def obtain_image_median(collection, time_range, area):
    """ Selection of median from a collection of images in the Earth Engine library
    See also: https://developers.google.com/earth-engine/reducers_image_collection

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        image_median (ee.image.Image)
     """
    collection = ee.ImageCollection(collection)

    ## Filter by time range and location
    collection_time = collection.filterDate(time_range[0], time_range[1])
    image_area = collection_time.filterBounds(area)
    image_median = image_area.median()
    return image_median

'''
The function below has been modified to accept the cloud cover threshold as an input
'''
def obtain_image_sentinel(collection, time_range, area, clouds):
    """ Selection of median, cloud-free image from a collection of images in the Sentinel 2 dataset
    See also: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        sentinel_median (ee.image.Image)
     """
#First, method to remove cloud from the image
    def maskclouds(image):
        band_qa = image.select('QA60')
        cloud_mask = ee.Number(2).pow(10).int()
        cirrus_mask = ee.Number(2).pow(11).int()
        mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
            band_qa.bitwiseAnd(cirrus_mask).eq(0))
        return image.updateMask(mask).divide(10000)

    sentinel_filtered = (ee.ImageCollection(collection).
                         filterBounds(area).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', clouds)).
                         map(maskclouds))

    sentinel_median = sentinel_filtered.median()
    return sentinel_median

def get_region(geom):
    """Get the region of a given geometry, needed for exporting tasks.

    Parameters:
        geom (ee.Geometry, ee.Feature, ee.Image): region of interest

    Returns:
        region (list)
    """
    if isinstance(geom, ee.Geometry):
        region = geom.getInfo()["coordinates"]
    elif isinstance(geom, ee.Feature, ee.Image):
        region = geom.geometry().getInfo()["coordinates"]
    elif isinstance(geom, list):
        condition = all([isinstance(item) == list for item in geom])
        if condition:
            region = geom
    return region

def get_url(name, image, scale, region, filePerBand=False):
    """It will open and download automatically a zip folder containing Geotiff data of 'image'.
    Parameters:
        name -  a base name to use when constructing filenames.
        image (ee.image.Image): image to export
        scale (int): resolution of export in meters (e.g: 30 for Landsat)
        region (list): region of interest
        filePerBand - whether to produce a different GeoTIFF per band (boolean).
            Defaults to true. If false, a single GeoTIFF is produced and all
            band-level transformations will be ignored.

    Returns:
        path (str)


    If additional parameters are needed, see also:
    https://github.com/google/earthengine-api/blob/master/python/ee/image.py

    Args:
        params: An object containing visualization options with the following
          possible values:
        name -  a base name to use when constructing filenames.
        bands -  a description of the bands to download. Must be an array of
            dictionaries, each with the following keys:
          id -  the name of the band, a string, required.
          crs -  an optional CRS string defining the band projection.
          crs_transform -  an optional array of 6 numbers specifying an affine
              transform from the specified CRS, in the order: xScale, yShearing,
              xShearing, yScale, xTranslation and yTranslation.
          dimensions -  an optional array of two integers defining the width and
              height to which the band is cropped.
          scale -  an optional number, specifying the scale in meters of the
                 band; ignored if crs and crs_transform is specified.
        crs -  a default CRS string to use for any bands that do not explicitly
            specify one.
        crs_transform -  a default affine transform to use for any bands that do
            not specify one, of the same format as the crs_transform of bands.
        dimensions -  default image cropping dimensions to use for any bands
            that do not specify them.
        scale -  a default scale to use for any bands that do not specify one;
            ignored if crs and crs_transform is specified.
        region -  a polygon specifying a region to download; ignored if crs
            and crs_transform is specified.
        filePerBand - whether to produce a different GeoTIFF per band (boolean).
            Defaults to true. If false, a single GeoTIFF is produced and all
            band-level transformations will be ignored.
     """
    path = image.getDownloadURL({
        'name':(name),
        'scale': scale,
        'region':(region),
        'filePerBand': (filePerBand)
        })

    webbrowser.open_new_tab(path)
    return path


# Authenticate to the Google Earth Engine API.

API stands for 'application programming interface'. An API defines interactions between multiple software intermediaries, in this case between our Jupyter Notebook and the ESA Copernicus Data Hub. It defines the kinds of calls or requests that can be made, how to make them, the data formats that should be used, the conventions to follow etc. (text modified after Wikipedia)

In [ ]:
# Connect to Google Earth Engine API
# This will open a web page where you have to enter your account information and a code is provided. Paste it in the terminal.
!earthengine authenticate

ee.Initialize()

Get some information about our shapefile.

In [ ]:
# Get the shapefile layer's extent
driver = ogr.GetDriverByName("ESRI Shapefile")
ds = driver.Open(shapefile, 0)
lyr = ds.GetLayer()
extent = lyr.GetExtent()
print("Extent of the area of interest (shapefile):\n", extent)
print(type(extent))

# get projection information of the shapefile
outSpatialRef = lyr.GetSpatialRef().ExportToWkt()
ds = None # close file
print("\nSpatial referencing information of the shapefile:\n", outSpatialRef)


Get the extent of the shapefile into a format that Google Earth Engine understands.

Look at the printed outputs of the type conversions. The code will make more sense then.

In [ ]:
# GEE needs a special format for defining an area of interest. 
# It has to be a GeoJSON Polygon and the coordinates should be first defined in a list and then converted using ee.Geometry. 
extent_list = list(extent)
print(extent_list)
print(type(extent_list))
# close the list of polygon coordinates by adding the starting node at the end again
# and make list elements in the form of coordinate pairs (y,x)
area_list = list([(extent[0], extent[2]),(extent[1], extent[2]),(extent[1], extent[3]),(extent[0], extent[3]),(extent[0], extent[2])])
print(area_list)
print(type(area_list))

search_area = ee.Geometry.Polygon(area_list)
print(search_area)
print(type(search_area))

Now we can access the Sentinel-2 collection on Google Earth Engine and run our search. This will return a URL (web link) from which we can download the data.

In [ ]:
# Obtain download links for image composites from an image collection on Google Earth Engine
# All products available are detailed on this page https://developers.google.com/earth-engine/datasets/.

# Name of the Sentinel 2 image collection
s2collection = ('COPERNICUS/S2')

# do the search on Google Earth Engine
s2median = obtain_image_sentinel(s2collection, time_range, search_area, clouds)

# to save disk space, we may want to download only certain bands
# band names for download, a list of strings
# only download R,G,B and NIR bands
bands = ['B2', 'B3', 'B4', 'B8']
print(bands)

# spatial resolution of the downloaded data
resolution = 20 # in units of metres

# Download images in Geotiff, using the get_url(name, image, scale, region) method
# ‘region’ is obtained from the area, but the format has to be adjusted using get_region(geom) method
search_region = get_region(search_area)
s2url = get_url('s2', s2median.select(bands), resolution, search_region, filePerBand=False)
print(s2url)

# Download the data

The next cell downloads the image composite as a zip file and unzips it.

In [ ]:
# change directory to download directory
os.chdir(downloaddir)

# request information on the file to be downloaded
f = requests.get(s2url, stream =True)

# check whether it is a zip file
check = zipfile.is_zipfile(io.BytesIO(f.content))

# either download the file as is, or unzip it
while not check:
    f = requests.get(s2url, stream =True)
    check = zipfile.is_zipfile(io.BytesIO(f.content))
else:
    z = zipfile.ZipFile(io.BytesIO(f.content))
    z.extractall()

# Explore the data directory structure of our downloaded files


In [ ]:
# where we stored the downloaded Sentinel-2 images
os.chdir(downloaddir)
print("contents of ", downloaddir, ":")
!ls -l

You should see the downloaded file.

Remember that we have saved the downloaded images to a temporary directory that will be deleted when we close the virtual machine. If you want to save your images to your local directory, this is how it goes.

Go to your Google Colab  folder in the panel on the left hand side.

Find the download directory and click on a Sentinel-2 image folder.

Right-click on it and select 'download' to save it.

# Show the image as a true colour composite

A true colour composite is a visualisation where the red, green and blue channels of the sensors are shown in the same colour on screen. Let's visualise our data composite in this way.

First, let's see what tiff files are in our directory.


In [ ]:
# get list of all tiff files in the directory
allfiles = [f for f in listdir(downloaddir) if isfile(join(downloaddir, f))]
print(allfiles)

# select the file for visualisation
thisfile = allfiles[0]
print(thisfile)

Now we know which image files we want to show on screen, the rest is easy. Just like last week.

Use our handy plotting function.
We modify it such that it is able to exclude the lowest and highest pixel values from each band separately using the NumPy percentile function as an option.

In [ ]:
def tci(afile, ax=None, bands=[3,2,1], percentiles=[0,100], xlim=None, ylim=None): 
  # tci stands for true colour image
  # afile is a handle to an image file opened with RasterIO.Open()
  # ax is the axes handle to plot the map on
  # bands is the order of image bands in the source file to become RGB channels
  # percentiles = list of percentiles for trimming the histogram
  #    [0,100] stands for min, max
  # xlim =[xmin, xmax] is the map extent to be shown in x direction
  # ylim =[ymin, ymax] is the map extent to be shown in y direction
  
  # we define a function within this function:
  def scale_to_uint8(x, percentiles=[0,100]):
    # scale array x to 0-255 and convert to uint8
    # x = input array
    # percentiles = list of percentiles for trimming the histogram
    #    [0,1] stands for min, max
    x = np.float32(x)
    amin = np.percentile(x, percentiles[0])
    amax = np.percentile(x, percentiles[1])
    anewmin = 0.0
    anewmax = 255.0
    xscaled = (x - amin) * ((anewmax - anewmin) / (amax - amin)) + anewmin
    return(xscaled.astype(np.uint8))

  # save the uint8 image as a temporary Geotiff file
  tmpfile = rasterio.open('tmp_rgb_imagefile_ cjdlsbYFEOGFHEWBVUW.tiff',
                            'w',driver='Gtiff', width=afile.width, height=afile.height,
                            count=3, crs=afile.crs, transform=afile.transform, 
                            dtype=np.uint8)

  # mask out extreme values for each band
  for b in range(3):
    # read band data
    a = afile.read(bands[b])
    a_uint8 = scale_to_uint8(a, percentiles) 
    # write the output into the new file as band b+1
    tmpfile.write(a_uint8, b+1)

  # close the file
  tmpfile.close()

  # try plotting the image
  imgfile = rasterio.open(r'tmp_rgb_imagefile_ cjdlsbYFEOGFHEWBVUW.tiff', count=3)

  if (xlim==None):
    xlim=[afile.bounds.left, afile.bounds.right]
    # afile.bounds returns a BoundingBox(left, bottom, right, top) object,
    #    from which we need to get the corner coordinates like so

  if (ylim==None):
    ylim=[afile.bounds.bottom, afile.bounds.top]
  
  # zoom in to an area of interest by setting the axes limits of our map
  ax.set_xlim(xlim)
  ax.set_ylim(ylim)
  plot.show(imgfile, ax=ax)

  # close the temporary file
  imgfile.close()

  # and remove the temporary file when we do not need it anymore
  os.remove('tmp_rgb_imagefile_ cjdlsbYFEOGFHEWBVUW.tiff')

  return()

The visualisation function is now defined and Python understands it when we call it. Now we can execute it and show our downloaded data on screen.

In [ ]:
# open file
f = rasterio.open(thisfile, 'r') 

# create a figure with 2x3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(10,16))
fig.patch.set_facecolor('white')

# the downloaded file is float32 data format
# for plotting, we need uint8 data format

# plot the image with full extent
tci(f, ax=ax1, percentiles=[0,98])

# zoom in to an area of interest
xlim=[-0.75, -0.70] # longitude coordinates
ylim=[52.66, 52.68] # latitude coordinates
tci(f, ax=ax2, percentiles=[0,98], xlim=xlim, ylim=ylim)

# zoom in elsewhere
xlim=[-0.70, -0.60]
ylim=[52.63, 52.68]
tci(f, ax=ax3, percentiles=[0,98], xlim=xlim, ylim=ylim)

Users of data often want the results in their own geographic projection.

Let's warp the images to the same projection as the shapefile. 

Remember we did this last week:

```
ds = gdal.Warp('Sentinel-2_stack_100m_BNG.tiff',
               'Sentinel-2_stack_100m.tiff', dstSRS='EPSG:27700')
ds = None #remember to close and save the output file
```

So let's put GDAL to work.



In [ ]:
# get the spatial referencing system of our shapefile into which we want to reproject the TCI images
# remember, we did this when we opened the shapefile earlier and saved it in outSpatialRef
print("Reprojecting image to the following projection:")
print(outSpatialRef)

# make a file name for our new file
warpfile = thisfile.split(sep='.')[0] + '_warped.tif'

# check whether the warp file already exists and skip if it does
if not os.path.exists(warpfile):
  # call the GDAL Warp command
  ds = gdal.Warp(warpfile, thisfile, dstSRS=outSpatialRef)
  ds = None #remember to close and save the output file
else:
  print("warped file already exists")

# Plot the shapefile on top of the raster

Suppose we want to see the locations of our polygons on top of our image composite. We can do that with the Geopandas library.

In [ ]:
# open the warped image
f = rasterio.open(warpfile, 'r') 

# create a figure with 2x3 subplots
fig, ax = plt.subplots(3,1, figsize=(10,16))
fig.patch.set_facecolor('white')

# Remember, the warped file is float32 data format
# for plotting, we need uint8 data format

# plot the image with full extent
tci(f, ax=ax[0], percentiles=[0,98])

# zoom in to an area of interest
xlim=[-0.75, -0.70]
ylim=[52.66, 52.68]
tci(f, ax=ax[1], percentiles=[0,98], xlim=xlim, ylim=ylim)

# zoom in elsewhere
xlim=[-0.70, -0.60]
ylim=[52.63, 52.68]
tci(f, ax=ax[2], percentiles=[0,98], xlim=xlim, ylim=ylim)

# We will use the Geopandas library for plotting the shapefile on top of the raster image.
shp = gpd.read_file(shapefile)
# add the shapefile to all three images
for i in range(3):
  shp.plot(ax=ax[i], facecolor="none", edgecolor="yellow")
  # set a title for the subplot
  mytitle = "Title"
  ax[i].set_title(mytitle, fontsize=8)

# Make a movie from several Sentinel-2 image composites

To analyse several images, we can simply repeat the API query and download temporal composites. These are made automatically by Google Earth Engine. In our case, we want to calculate the median reflectance of all pixel values that are cloud-free, aggregated by month.

For this task, we copy and paste the code from above into a single cell (below), and iterate over the different months for our searches. The for loop does the job for us.

We will use the imageio library to make a movie from the results.

In [ ]:
# Obtain monthly image composites

# change directory to download directory
os.chdir(downloaddir)

# make a list of lists with all date ranges for our new searches
months = [['2020-01-01', '2020-01-31'],
          ['2020-02-01', '2020-02-29'],
          ['2020-03-01', '2020-03-31'],
          ['2020-04-01', '2020-04-30'],
          ['2020-05-01', '2020-05-31'],
          ['2020-06-01', '2020-06-30'],
          ['2020-07-01', '2020-07-31'],
          ['2020-08-01', '2020-08-31'],
          ['2020-09-01', '2020-09-30'],
          ['2020-10-01', '2020-10-31']]

# set cloud cover threshold
clouds = 30

# band names for download, a list of strings
# only download R,G,B and NIR bands
bands = ['B2', 'B3', 'B4', 'B8']

# spatial resolution of the downloaded data
resolution = 20 # in units of metres

# iterate over the months
for month in range(len(months)):
  time_range = months[month]
  print(time_range)

  # do the search on Google Earth Engine
  s2median = obtain_image_sentinel(s2collection, time_range, search_area, clouds)
  print(type(s2median))

  # print out the band names of the image composite that was returned by our search
  band_names = s2median.bandNames().getInfo()
  print(band_names)

  # check whether the search returned any imagery
  if len(band_names) == 0:

    print("Search returned no results.")

  # if there are band names in our search results, proceed
  else:
    
    # begin the file name with this ID
    file_id = 's2_month'
    
    s2url = get_url(file_id+"{0:3d}".format(month+1), s2median.select(bands), resolution, search_region, filePerBand=False)
    print(s2url)

    # request information on the file to be downloaded
    f = requests.get(s2url, stream =True)

    # check whether it is a zip file
    check = zipfile.is_zipfile(io.BytesIO(f.content))

    # either download the file as is, or unzip it
    while not check:
        f = requests.get(s2url, stream =True)
        check = zipfile.is_zipfile(io.BytesIO(f.content))
    else:
        z = zipfile.ZipFile(io.BytesIO(f.content))
        z.extractall()

# after downloading all image composites, get a list of all files we want to warp
allfiles = [f for f in listdir(downloaddir) if isfile(join(downloaddir, f))]
files_for_warp = [s for s in allfiles if file_id in s]
print("Files for warping:")
pprint(sorted(files_for_warp))

# now warp them all
for f in files_for_warp:
  # make a file name for our new file
  warpfile = f.split('.')[0]+'_warped.tif'
  # call the GDAL Warp command
  ds = gdal.Warp(warpfile, f, dstSRS=outSpatialRef)
  ds = None #remember to close and save the output file

Get all images into the same projection as the shapefile using GDAL warp.

Then save them as uint8 format for making the movie.

In [ ]:
# after downloading and warping all image composites, get a list of all warped tiff files in the directory
allfiles = [f for f in listdir(downloaddir) if isfile(join(downloaddir, f))]
warpfiles = [s for s in allfiles if "_warped.tif" in s]
print("Files after warping:")
pprint(sorted(warpfiles))

# Remember, the warped file is float32 data format
# for plotting, we need uint8 data format
# We can adapt our previous tci function and take out the plotting but save the image data
def save_as_uint8(afile, outfile, bands=[3,2,1], percentiles=[0,100]):
  # afile is a handle to an image file opened with RasterIO.Open()
  # outfile is the output file name
  # percentiles = list of percentiles for trimming the histogram
  #    [0,100] stands for min, max
  # bands is the order of image bands in the source file to become RGB channels

  def scale_to_uint8(x, percentiles=[0,100]):
    # scale array x to 0-255 and convert to uint8
    # x = input array
    # percentiles = list of percentiles for trimming the histogram
    #    [0,1] stands for min, max
    x = np.float32(x)
    amin = np.percentile(x, percentiles[0])
    amax = np.percentile(x, percentiles[1])
    anewmin = 0.0
    anewmax = 255.0
    xscaled = (x - amin) * ((anewmax - anewmin) / (amax - amin)) + anewmin
    return(xscaled.astype(np.uint8))

  # save the uint8 image as a temporary Geotiff file
  outf = rasterio.open(outfile,'w',driver='Gtiff', width=afile.width, height=afile.height,
                       count=3, crs=afile.crs, transform=afile.transform, dtype=np.uint8)

  # mask out extreme values for each band
  for b in range(3):
    # read band data
    a = afile.read(bands[b])
    a_uint8 = scale_to_uint8(a, percentiles) 
    # write the output into the new file as band b+1
    outf.write(a_uint8, b+1)

  # close the file
  outf.close()

  return()

# iterate over all warpfiles
for w in sorted(warpfiles):
  print(w)
  wfile=rasterio.open(w, 'r')
  # make a file name for our new file
  outfile = w.split(sep='.')[0] + '_uint8.tif'
  save_as_uint8(wfile, outfile)

# get a list of all warped tiff files in uint8 data format in the directory
allfiles = [f for f in listdir(downloaddir) if isfile(join(downloaddir, f))]
uint8files = [s for s in allfiles if "_warped_uint8.tif" in s]
pprint(sorted(uint8files))

Now it is time to make the actual movie.

In [ ]:
import imageio
# create an empty Numpy array where we will merge all raster images
images = []
# iterate over all zoom files
for f in sorted(uint8files):
  images.append(imageio.imread(f)) # read the next image and append it

# Let's set the frame rate to 3 seconds.
framerate = { 'duration': 3 }

# save the movie
imageio.mimsave(join(downloaddir, "movie.gif"), images, **framerate)


Now download the file movie.gif from Colab using the folder icon on the left hand side. Locate the file in the 'out' directory, right-click and select 'download'. 

Save it to your local hard drive and open it with Google Chrome to view it.